In [6]:
# ! pip install -r requirements.txt
! pip freeze | grep psycopg

psycopg2-binary==2.9.6


In [17]:
import sqlalchemy

In [23]:
! python3 db/random_generator.py

Adding Data...
Create Table Query: SQL('CREATE TABLE IF NOT EXISTS dummy_table_2 (\n    id SERIAL PRIMARY KEY,\n    name varchar(255),\n\tage integer,\n\tgender varchar(255)\n);\n')
None
Creating Statements...
Created, Inserting...
Inserted, closing connection...
Done.
Added 100 rows in  25.335041761398315 seconds.


In [1]:
from db.source.psql_source import get_postgres_datatype

'text'

In [16]:
! pip install faker

     |████████████████████████████████| 1.7 MB 1.1 MB/s eta 0:00:01
You should consider upgrading via the '/Users/atharva/de/venv/bin/python3 -m pip install --upgrade pip' command.


In [17]:
# ! pip freeze 

     |████████████████████████████████| 53 kB 865 kB/s eta 0:00:011
     |████████████████████████████████| 105 kB 1.9 MB/s eta 0:00:01
You should consider upgrading via the '/Users/atharva/de/venv/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import os
import ssl
from dotenv import find_dotenv, load_dotenv

dotenv_path = find_dotenv()
load_dotenv(dotenv_path)

import sqlalchemy
from sqlalchemy import create_engine




def connect_tcp_socket() -> sqlalchemy.engine.base.Engine:
    """Initializes a TCP connection pool for a Cloud SQL instance of Postgres."""
    db_host = os.environ.get("SOURCE_HOST")
    db_user = os.environ.get("SOURCE_USER")  # e.g. 'my-db-user'
    db_pass = os.environ.get("SOURCE_PASS")  # e.g. 'my-db-password'
    db_name = os.environ.get("SOURCE_DB")  # e.g. 'my-database'
    db_port = os.environ.get("SOURCE_PORT")  # e.g. 5432

    pool = sqlalchemy.create_engine(
        # Equivalent URL:
        # postgresql+pg8000://<db_user>:<db_pass>@<db_host>:<db_port>/<db_name>
        sqlalchemy.engine.url.URL.create(
            drivername="postgresql+pg8000",
            username=db_user,
            password=db_pass,
            host=db_host,
            port=db_port,
            database=db_name,
        ),
        # ...
    )
    return pool


In [15]:
engine = connect_tcp_socket()

In [14]:
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.orm import sessionmaker, declarative_base
from faker import Faker
import random

In [15]:
Base = declarative_base()

class DummyTable(Base):
    __tablename__ = 'dummy_table'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    age = Column(Integer)
    gender = Column(String)

In [17]:
engine = connect_tcp_socket()
Base.metadata.create_all(engine)

In [18]:
import psycopg2
from faker import Faker
import random
import os
from dotenv import find_dotenv, load_dotenv

dotenv_path = find_dotenv()
load_dotenv(dotenv_path)

def psycopg_client():
    db_url = f"dbname={os.environ.get('SOURCE_DB')} user={os.environ.get('SOURCE_USER')} password={os.environ.get('SOURCE_PASS')} host={os.environ.get('SOURCE_HOST')}"
    return psycopg2.connect(db_url)

def generate_dummy_data(num_rows):
    fake = Faker()
    dummy_data = []
    for _ in range(num_rows):
        dummy_data.append({
            'name': fake.name(),
            'age': random.randint(18, 99),
            'gender': random.choice(['M', 'F'])
        })
    return dummy_data

def insert_dummy_data(data,engine):
    Session = sessionmaker(bind=engine)
    session = Session()
    for row in data:
        dummy_row = DummyTable(**row)
        session.add(dummy_row)
    session.commit()
    session.close()
    
def insert_dummy_data_fast(data, table_name,conn):
    with conn.cursor() as cursor:
        insert_query = """
            INSERT INTO {table_name}
            VALUES %s;
        """
        # Convert the dictionaries to tuples
        data_tuples = [(row['name'], row['age'], row['gender']) for row in data]
        
        # Execute the insert query with multiple rows
        psycopg2.extras.execute_values(cursor, insert_query, data_tuples)
        conn.commit()

In [21]:
dummy_data = generate_dummy_data(100)

In [4]:
import os
import ssl
from dotenv import find_dotenv, load_dotenv

dotenv_path = find_dotenv()
load_dotenv(dotenv_path)

import sqlalchemy
from sqlalchemy import create_engine




def connect_tcp_socket() -> sqlalchemy.engine.base.Engine:
    """Initializes a TCP connection pool for a Cloud SQL instance of Postgres."""
    db_host = os.environ.get("SOURCE_HOST")
    db_user = os.environ.get("SOURCE_USER")  # e.g. 'my-db-user'
    db_pass = os.environ.get("SOURCE_PASS")  # e.g. 'my-db-password'
    db_name = os.environ.get("SOURCE_DB")  # e.g. 'my-database'
    db_port = os.environ.get("SOURCE_PORT")  # e.g. 5432

    pool = sqlalchemy.create_engine(
        # Equivalent URL:
        # postgresql+pg8000://<db_user>:<db_pass>@<db_host>:<db_port>/<db_name>
        sqlalchemy.engine.url.URL.create(
            drivername="postgresql+pg8000",
            username=db_user,
            password=db_pass,
            host=db_host,
            port=db_port,
            database=db_name,
        ),
        # ...
    )
    return pool


In [6]:
engine = connect_tcp_socket()
from sqlalchemy import MetaData, Table
metadata = MetaData()
metadata.reflect(bind=engine,schema='public')


In [7]:
print([t for t in metadata.tables.keys()])

['public.dummy_table']


In [22]:
insert_dummy_data(dummy_data,engine)

NameError: name 'engine' is not defined

In [32]:
cursor = engine.connect()

InterfaceError: (pg8000.exceptions.InterfaceError) Can't create a connection to host 34.131.22.145 and port 5432 (timeout is None and source_address is None).
(Background on this error at: https://sqlalche.me/e/20/rvf5)